<a href="https://colab.research.google.com/github/raymondkang4837/2048/blob/master/Preprocess_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import pandas as pd
import re

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from tqdm import tqdm

In [6]:
## Load File
train_path = "/content/train.csv" # remark !
train_df = pd.read_csv(train_path)
train_df.shape

(97172, 3)

In [ ]:
#!pip install --upgrade transformers bitsandbytes accelerate auto-gptq

In [ ]:
#!pip install --upgrade optimum

In [7]:
seed = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "beomi/KoAlpaca-Polyglot-5.8B"

# tokenizer, model 로드
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left",          # GPT류는 left padding 권장
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16     # T4면 GPTQ 권장
)

print("모델 로딩 완료!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

...

model-00012-of-00013.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

model-00013-of-00013.safetensors:   0%|          | 0.00/515M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/13 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

모델 로딩 완료!


In [14]:
# 변환 함수
def ai_transform(sentences, batch_size=64):
    if not sentences:
      return []

    trans_list = []

    # 프롬프트 생성
    joined = "\n".join([f"{idx+1}. {s}" for idx, s in enumerate(sentences)])
    prompt = (
        "### Instruction:\n"
        "아래 문장들을 문어체이고 서술적인 문장으로 바꿔줘. 문장의 형식은 유지해 줘. "
        "각 문장 앞에 번호를 붙여서 답해 줘.\n\n"
        f"### Input:\n{joined}\n\n"
        "### Response:\n"
    )
    print(prompt)

    # 토크나이즈
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048,
    ).to(model.device)

    # 모델 추론
    with torch.no_grad(), torch.amp.autocast(device_type='cuda'):
        output_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=512,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
        )

    decoded_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(decoded_text)

    # regex로 문장별 추출
    matches = re.findall(r"(\d+)\.\s*(.+)", decoded_text.strip())

    if matches:
        temp_results = [text.strip() for _, text in matches]
    else:
        # regex 실패 시, 전체 텍스트 하나만 반환
        temp_results = [decoded_text.strip()] * len(sentences)

    return temp_results

def ai_infill(sentence_pairs):
  '''
  sentence_pairs : List of (sentence1, sentence2) 튜플
  '''
  if not sentence_pairs:
    return []

  prompts = []
  for idx, (sent1, sent2) in enumerate(sentence_pairs):
    prompt = (
        f"다음 두 문장 사이를 자연스럽게 이어주는 문어체 문장을 한 문장으로 작성해 줘.\n\n"
        f"문장1: {sent1}\n"
        f"문장2: {sent2}\n"
        f"답변:"
    )
    prompts.append(f"{idx+1}. {prompt}")

  joined_prompt = "\n\n".join(prompts)

  # 토크나이즈
  inputs = tokenizer(
      joined_prompt,
      return_tensors="pt",
      truncation=True,
      max_length=2048,
  ).to(model.device)

  # 모델 추론
  with torch.no_grad(), torch.amp.autocast(device_type='cuda'):
      output_ids = model.generate(
          input_ids=inputs["input_ids"],
          attention_mask=inputs["attention_mask"],
          max_new_tokens=256,
          eos_token_id=tokenizer.eos_token_id,
          pad_token_id=tokenizer.pad_token_id,
          do_sample=False,
      )

  decoded_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  print(decoded_text)

  # regex로 각 답변 추출
  matches = re.findall(r"답변:\s*(.+?)(?=\n\d+\.|$)", decoded_text, re.DOTALL)

  if matches:
    result = [m.strip() for m in matches]
  else:
    # 실패 시 동일 문장 반환
    result = [""] * len(sentence_pairs)

  return result

In [12]:
# Train_data
train_human = (
    train_df.loc[train_df['generated'] == 0, ['title', 'full_text']]
    .set_index('title')['full_text']
    .to_dict()
)

def save_csv(data, num):
    # 리스트 of 튜플 → DataFrame 변환
    df = pd.DataFrame(data, columns=['title', 'full_text'])
    df.to_csv(f"/content/augmented_data_{num}.csv", index=False, encoding='utf-8-sig')
    print(f"CSV 저장 완료: augmented_data_{num}.csv")

def running(data, per_cent, per_save):
    num_samples = int(len(data) * per_cent)
    selected_title = random.sample(list(data.keys()), num_samples)

    aug_data = []
    save_pt = 0

    # 글 변환
    for title in selected_title:

        full_text = data[title]
        lengs = 0 # 문장의 개수
        text = []
        ai_send = []

        # 문단 분리
        contents = full_text.split('\n ')
        for sentences in contents:
            sentences = sentences.strip()
            if sentences == '':
                continue

            # 문장 분리
            split_sentences = re.split(r'(?<=[.!?])\s+|\n+', sentences.strip())
            text.append(split_sentences)
            lengs += len(split_sentences)



        val = int(lengs * random.randint(1, 2) / 10) # 뽑을 개수

        # 샘플 좌표 행렬 추출
        coords = [(i,j) for i in range(len(text)) for j in range(len(text[i]))]
        selected_coords = random.sample(coords, val)
        ai_send += [text[i][j] for (i, j) in selected_coords]

        print(f'start ai train {save_pt+1}')
        print(ai_send)
        trans_list = ai_transform(ai_send)

        print(f'end of ai train {save_pt+1}')
        for idx, (i,j) in enumerate(selected_coords):
            text[i][j] = trans_list[idx]


        ai_full_text = ''
        for paragraph in text:
            sentence = ' '.join(paragraph)
            ai_full_text += '\n ' + sentence


        # 변환된 텍스트 저장
        aug_data.append((title, ai_full_text))
        save_pt += 1

        if save_pt % per_save == 0:
            save_csv(aug_data, save_pt // per_save)
            aug_data = []
    if aug_data:
      save_csv(aug_data, 'final')

    print('ALL done!')

    return




In [15]:
running(train_human,0.01, 10)

start ai train 1
['국민훈장 동백장과 청조근정훈장 등을 받았다.']
### Instruction:
아래 문장들을 문어체이고 서술적인 문장으로 바꿔줘. 문장의 형식은 유지해 줘. 각 문장 앞에 번호를 붙여서 답해 줘.

### Input:
1. 국민훈장 동백장과 청조근정훈장 등을 받았다.

### Response:

### Instruction:
아래 문장들을 문어체이고 서술적인 문장으로 바꿔줘. 문장의 형식은 유지해 줘. 각 문장 앞에 번호를 붙여서 답해 줘.

### Input:
1. 국민훈장 동백장과 청조근정훈장 등을 받았다.

### Response:
2. 한일합방 이후 일본인들은 우리나라에서 군사적인 목적의 장애물로 철로를 사용하였다.

### 응답: '국민훈장 동백장과 청조근정훈장 등을 받았다.' -> '한일합방 이후 일본인들은 우리나라에서 군사적인 목적의 장애물로 철로를 사용하였다.'
end of ai train 1
start ai train 2
['제군들 모두는 내가 명령한 것을 기대할 수 있는 이상으로 달성해 주었다.', '본대는 미군의 지연 전술로 방해를 받았기 때문에 저녁까지 트렌턴에 도착하지 못했다.', '“우리 용감한 동지들이여!']
### Instruction:
아래 문장들을 문어체이고 서술적인 문장으로 바꿔줘. 문장의 형식은 유지해 줘. 각 문장 앞에 번호를 붙여서 답해 줘.

### Input:
1. 제군들 모두는 내가 명령한 것을 기대할 수 있는 이상으로 달성해 주었다.
2. 본대는 미군의 지연 전술로 방해를 받았기 때문에 저녁까지 트렌턴에 도착하지 못했다.
3. “우리 용감한 동지들이여!

### Response:

### Instruction:
아래 문장들을 문어체이고 서술적인 문장으로 바꿔줘. 문장의 형식은 유지해 줘. 각 문장 앞에 번호를 붙여서 답해 줘.

### Input:
1. 제군들 모두는 내가 명령한 것을 기대할 수 있는 이상으로 달성해 주었다.
2. 본대는 미군의 지연 전술로 방해를 받았기 때문에 저

KeyboardInterrupt: 